<a href="https://colab.research.google.com/github/mounikamarreddy/BERTFinetuning-NLPTasks/blob/main/Bert_AbstractiveSummarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install tensorflow
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.6 MB/s 
     |████████████████████████████████| 120 kB 66.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.7 MB/s 
     |████████████████████████████████| 115 kB 65.6 MB/s 
     |████████████████████████████████| 212 kB 59.5 MB/s 
     |████████████████████████████████| 127 kB 57.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datasets import load_dataset
import re
import transformers
from transformers import BertTokenizer, BertModel, EncoderDecoderModel
import torch
from tqdm import tqdm_notebook as tqdm
import datasets
from transformers import TrainingArguments
from transformers import Trainer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional
from difflib import SequenceMatcher

In [3]:
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text

In [4]:
train_data = load_dataset("cnn_dailymail","3.0.0",split="train[:1%]")
#dataset = load_dataset("cnn_dailymail","3.0.0")
val_data = load_dataset("cnn_dailymail","3.0.0",split="validation[:1%]")
test_data = load_dataset("cnn_dailymail","3.0.0",split="test[:1%]")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


In [5]:
#data preprocessing
def load_data(path):
    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    print('found {} files'.format(len(onlyfiles)))
    all_text = []
    for f in onlyfiles:
        with open('{}/{}'.format(path, f)) as handle:
            lines = clean_text(handle.readlines()[0])
            all_text.append(lines)
        
    return all_text

In [6]:
# The following function iterates through all the articles given and creates a list of overlapping sections for each article 
# if that article is greater than 512 words  
def overlapping_subsection(dataset):
    articles = []
    for article in tqdm(dataset):
        article = article.split()
        word_count = 0
        article_subsections = []
        while len(article) > 512:
            if len(article_subsections) == 0:
                article_subsections.append(' '.join(word for word in article[:512]))
                word_count = 512
                article = article[word_count-100:]       
            if len(article) > 412:
                article_subsections.append(' '.join(word for word in article[:512]))   
                word_count = word_count + 412
                article = article[412:]
            if len(article) < 412:
                article_subsections.append(' '.join(word for word in article))   
        if len(article_subsections) != 0: 
            article = article_subsections
        articles.append(article)
    return articles

In [7]:
# The following is the orginal data after running it through the overlapping function saved as a pandas DataFrame
train_df =  pd.DataFrame({'article':overlapping_subsection(train_data['article']),'highlights':train_data['highlights'],"id": train_data['id']})
test_df = pd.DataFrame({'article':overlapping_subsection(test_data['article']),'highlights':test_data['highlights'],"id": test_data['id']})
val_df =  pd.DataFrame({'article':overlapping_subsection(val_data['article']),'highlights':val_data['highlights'],"id": val_data['id']})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2871 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

In [8]:
#The tokenizer to be used to create embeddings for both Articles and Summaries
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
batch_size = 2 # Change batch size to 4 for faster but less accurate training
encoder_max_length = 512
decoder_max_length = 128

def convert_data_to_model_inputs(batch):
    #Encodes the article
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length = encoder_max_length)
    #Encodes the summary
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length = decoder_max_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]
    
    return batch
    

In [10]:
#setup training
#Note: If data has already been mapped and saved to file DO NOT run this cell

# Uncomment out the following two line two train on a small subset
# train_data = train_data.select(range(32))
# val_data = val_data.select(range(32))

train_data = train_data.map(
    convert_data_to_model_inputs,
    batched = True,
    batch_size = batch_size,
    remove_columns=["article","highlights", "id"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)
val_data = val_data.map(
    convert_data_to_model_inputs,
    batched = True,
    batch_size = batch_size,
    remove_columns = ["article","highlights", "id"]
)
val_data.set_format(
    type="torch", columns = ["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/1436 [00:00<?, ?ba/s]

  0%|          | 0/67 [00:00<?, ?ba/s]

In [11]:
#saves mapped data to files
# The following saves the training and validation data to file
# File size < 1.75 Gb
import pickle
with open('train_data.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('val_data.pickle', 'wb') as handle:
    pickle.dump(val_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
#Load mapped data from the files
#The following loads pre-mapped train/validation data
with open('train_data.pickle', 'rb') as handle:
     train_data = pickle.load(handle)
        
with open('val_data.pickle', 'rb') as handle:
    val_data = pickle.load(handle)

In [13]:
#Creates the base encoder decoder model
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased","bert-base-uncased")

# set model configuration
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
model.config.vocab_size = model.config.decoder.vocab_size
model.config.max_length = 128
model.config.min_length = 64
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [14]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=34bc99b48662ba824b264674703c99b93aef7014004061719857a74709c3f1fb
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [15]:
import datasets
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output1 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge1_precision": round(rouge_output1.precision, 4),
        "rouge1_recall": round(rouge_output1.recall, 4),
        "rouge1_fmeasure": round(rouge_output1.fmeasure, 4),
        "rouge2_precision": round(rouge_output2.precision, 4),
        "rouge2_recall": round(rouge_output2.recall, 4),
        "rouge2_fmeasure": round(rouge_output2.fmeasure, 4),
    }

In [17]:
#Frees up unused memory 
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    evaluation_strategy='steps',
    do_train=True,
    do_eval=True,
    logging_steps=1000,
    save_steps=500, 
    eval_steps=8000,
    warmup_steps=2000,  
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, #Comment out this line if training a non-CUDA device
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2871
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 4308


Step,Training Loss,Validation Loss


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
Deleting older checkpoint [checkpoint-70] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [checkpoint-10] due to args.save_total_limit
/usr

TrainOutput(global_step=4308, training_loss=0.0057959430544060365, metrics={'train_runtime': 1205.4787, 'train_samples_per_second': 7.145, 'train_steps_per_second': 3.574, 'total_flos': 5283680167434240.0, 'train_loss': 0.0057959430544060365, 'epoch': 3.0})

In [ ]:
#evaluate the model
model = EncoderDecoderModel.from_pretrained("./checkpoint-3000")
model.to("cuda") # Comment out this line if not using CUDA
batch_size = 64

def evaluate_test_data(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length = encoder_max_length,return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda") # Remove ".to("cuda")" if not using CUDA
    attention_mask = inputs.attention_mask.to("cuda") # Remove ".to("cuda")" if not using CUDA
    
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch


results = test_data.map(evaluate_test_data, batched=True, batch_size=batch_size, remove_columns=["article"])

pred_str = results["pred"]
label_str = results["highlights"]

rouge_output1 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output1)
print(rouge_output2)

loading configuration file ./checkpoint-3000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "bert-base-uncased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    }